In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sarscov2-fasta/nCoV.fasta
/kaggle/input/ncovwuhan1/nCoV-Wuhan.fasta
/kaggle/input/ncovusa/nCoV-USA.fasta


In [2]:
from Bio import SeqIO
for seq_record in SeqIO.parse('../input/ncovwuhan1/nCoV-Wuhan.fasta', "fasta"):
    print(seq_record.id)
    print(seq_record.seq)

NC_045512.2
...


In [3]:
import numpy as np
import re
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

# create a label encoder with 'acgtn' alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [4]:
# function to encode a DNA sequence string as an ordinal vector
# returns a numpy vector with a=0.25, c=0.50, g=0.75, t=1.00, n=0.00
def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    return float_encoded

In [5]:
test_sequence = 'gtttatacct'
ordinal_encoder(string_to_array(test_sequence))

array([0.75, 1.  , 1.  , 1.  , 0.25, 1.  , 0.25, 0.5 , 0.5 , 1.  ])

In [6]:
# function to one-hot encode a DNA sequence string
# non 'acgt' bases (n) are 0000
# returns a L x 4 numpy array
from sklearn.preprocessing import OneHotEncoder
def one_hot_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)#n_values=n
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded

In [7]:
test_sequence = 'gtttatacct'
one_hot_encoder(string_to_array(test_sequence))

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [8]:
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [9]:
mySeq = 'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA'
getKmers(mySeq, size=6)

['attaaa',
 'ttaaag',
 'taaagg',
 'aaaggt',
 'aaggtt',
 'aggttt',
 'ggttta',
 'gtttat',
 'tttata',
 'ttatac',
 'tatacc',
 'atacct',
 'tacctt',
 'accttc',
 'ccttcc',
 'cttccc',
 'ttccca',
 'tcccag',
 'cccagg',
 'ccaggt',
 'caggta',
 'aggtaa',
 'ggtaac',
 'gtaaca',
 'taacaa',
 'aacaaa',
 'acaaac',
 'caaacc',
 'aaacca',
 'aaccaa',
 'accaac',
 'ccaacc',
 'caacca',
 'aaccaa',
 'accaac',
 'ccaact',
 'caactt',
 'aacttt',
 'actttc',
 'ctttcg',
 'tttcga',
 'ttcgat',
 'tcgatc',
 'cgatct',
 'gatctc',
 'atctct',
 'tctctt',
 'ctcttg',
 'tcttgt',
 'cttgta',
 'ttgtag',
 'tgtaga',
 'gtagat',
 'tagatc',
 'agatct',
 'gatctg',
 'atctgt',
 'tctgtt',
 'ctgttc',
 'tgttct',
 'gttctc',
 'ttctct',
 'tctcta',
 'ctctaa',
 'tctaaa']

In [10]:
words = getKmers(mySeq, size=6)
sentence = ' '.join(words)
sentence

'attaaa ttaaag taaagg aaaggt aaggtt aggttt ggttta gtttat tttata ttatac tatacc atacct tacctt accttc ccttcc cttccc ttccca tcccag cccagg ccaggt caggta aggtaa ggtaac gtaaca taacaa aacaaa acaaac caaacc aaacca aaccaa accaac ccaacc caacca aaccaa accaac ccaact caactt aacttt actttc ctttcg tttcga ttcgat tcgatc cgatct gatctc atctct tctctt ctcttg tcttgt cttgta ttgtag tgtaga gtagat tagatc agatct gatctg atctgt tctgtt ctgttc tgttct gttctc ttctct tctcta ctctaa tctaaa'

In [11]:
mySeq2 = 'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA'
mySeq3 = 'CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC'
sentence2 = ' '.join(getKmers(mySeq2, size=6))
sentence3 = ' '.join(getKmers(mySeq3, size=6))

In [12]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform([sentence, sentence2, sentence3]).toarray()

In [13]:
X

array([[0, 1, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 1, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0